In [36]:
import glob
import numpy as np

from astropy.io import fits

In [37]:
from photutils import CircularAnnulus as CAn
from photutils import aperture_photometry as apPho

# Overscan Removal

In [38]:
fits_files = [line.rstrip('\n') for line in open('data_files.list')]
os_removed = [line.rstrip('\n') for line in open('files_osr.list')]

In [39]:
n = len(fits_files)

for i in range(0,n):
    data = fits.getdata(fits_files[i])
    data = data[350:1650, 350:1650]
    fits.writeto(os_removed[i], data, overwrite=True)

# Bias Subtraction

In [40]:
bias_files = glob.glob('mid/osr_*bias*.fits')
bias_files

['mid/osr_arctic_bias.0001.fits',
 'mid/osr_arctic_bias.0002.fits',
 'mid/osr_arctic_bias.0003.fits',
 'mid/osr_arctic_bias.0004.fits',
 'mid/osr_arctic_bias.0005.fits']

In [41]:
data_in = [line.rstrip('\n') for line in open('osr_flats.list')]
data_out = [line.rstrip('\n') for line in open('bias_osr_flats.list')]
data_out

['5007/bias_osr_arctic_5007_flat.0001.fits',
 '5007/bias_osr_arctic_5007_flat.0002.fits',
 '5007/bias_osr_arctic_5007_flat.0003.fits',
 'r/bias_osr_arctic_m97.0001.fits',
 'r/bias_osr_arctic_m97.0002.fits',
 '5007/bias_osr_arctic_m97.0003.fits',
 '5007/bias_osr_arctic_m97.0004.fits',
 'r/bias_osr_arctic_r_flat.0003.fits',
 'r/bias_osr_arctic_r_flat.0004.fits',
 'r/bias_osr_arctic_r_flat.0005.fits']

In [42]:
data_stack = []
for file in bias_files:
    data_stack.append(fits.getdata(file))

median = np.median(data_stack, axis = 0)
header = fits.getheader(bias_files[0])
header['HISTORY'] = 'Median combined'

n = len(data_in)

for i in range(0,n):
    data = fits.getdata(data_in[i], header=False)
    data = data - median
    fits.writeto(data_out[i],data, overwrite=True)

# 5007 Filter

#### normalize and combine flats

In [43]:
flats_5007 = [line.rstrip('\n') for line in open('flats_5007.list')]
flats_5007

['5007/bias_osr_arctic_5007_flat.0001.fits',
 '5007/bias_osr_arctic_5007_flat.0002.fits',
 '5007/bias_osr_arctic_5007_flat.0003.fits']

In [44]:
flat_stack_5007 = []

for file in flats_5007:
    data = fits.getdata(file, header=False)
    data = data / np.median(data)
    flat_stack_5007.append(data)

flat_5007 = np.median(flat_stack_5007, axis=0)
mean = np.mean(flat_5007)
avg_5007 = flat_5007 / mean

fits.writeto('5007/flat_avg_5007.fits', avg_5007, overwrite=True)

In [45]:
image_5007_in = [line.rstrip('\n') for line in open('image_5007.list')]
image_5007_out = [line.rstrip('\n') for line in open('image_5007_out.list')]
image_5007_out

['5007/flattened_arctic_m97.0003.fits', '5007/flattened_arctic_m97.0004.fits']

In [46]:
n = len(image_5007_in)

for i in range(0,n):
    data = fits.getdata(image_5007_in[i], header=False)
    data = data / avg_5007
    fits.writeto(image_5007_out[i], data, overwrite=True)

#### Photometry

In [47]:
r_ap = 480 # all nebula
r_in = 487 # edge of nebula
r_out = 500 # background to subtract

position = [(630,639)]
annulus = CAn(position, r_in, r_out)

In [48]:
image_data = fits.getdata(image_5007_out[0])
annulus_center = apPho(image_data, annulus, method='center')

bkg_mean = annulus_center['aperture_sum'] / annulus.area()
bkg = np.ones((1300,1300))*bkg_mean

final_image = image_data - bkg

fits.writeto('final_arctic_m97.0003.fits', final_image, overwrite=True)

In [49]:
image_data = fits.getdata(image_5007_out[1])
annulus_center = apPho(image_data, annulus, method='center')

bkg_mean = annulus_center['aperture_sum'] / annulus.area()
bkg = np.ones((1300,1300))*bkg_mean

final_image = image_data - bkg

fits.writeto('final_arctic_m97.0004.fits', final_image, overwrite=True)

# R filter

In [50]:
flats_r = [line.rstrip('\n') for line in open('flats_r.list')]

In [51]:
flat_stack_r = []
for file in flats_r:
    data = fits.getdata(file, header=False)
    data = data / np.median(data)
    flat_stack_r.append(data)
    
flat_r = np.median(flat_stack_r, axis=0)
mean = (flat_r)
avg_r = flat_r / mean

fits.writeto('r/flat_avg_r.fits', avg_5007, overwrite=True)

In [52]:
image_r_in = [line.rstrip('\n') for line in open('image_r_in.list')]
image_r_out = [line.rstrip('\n') for line in open('image_r_out.list')]

In [53]:
n = len(image_r_in)

for i in range(0,n):
    data = fits.getdata(image_r_in[i], header=False)
    data = data / avg_r
    fits.writeto(image_r_out[i], data, overwrite=True)

#### Photometry

In [54]:
image_data = fits.getdata(image_r_out[0])
annulus_center = apPho(image_data, annulus, method='center')

bkg_mean = annulus_center['aperture_sum'] / annulus.area()
bkg = np.ones((1300,1300))*bkg_mean

final_image = image_data - bkg

fits.writeto('final_arctic_m97.0001.fits', final_image, overwrite=True)

In [55]:
image_data = fits.getdata(image_r_out[1])
annulus_center = apPho(image_data, annulus, method='center')

bkg_mean = annulus_center['aperture_sum'] / annulus.area()
bkg = np.ones((1300,1300))*bkg_mean

final_image = image_data - bkg

fits.writeto('final_arctic_m97.0002.fits', final_image, overwrite=True)